In [5]:
!pip install transformers
!pip install captum

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


After we pretrain the model, we can load the tokenizer and pre-trained BERT model.

In [9]:
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = '/content/drive/My Drive/Imperial/BERT-results/for-gradient-descent/model/102942_Y1Y2_ReasoningLevel_trainsize08_4epochs_16batchsize_1e-5lr'

# load model
model = BertForQuestionAnswering.from_pretrained(model_path)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

OSError: ignored

Tokenize text & create dataset

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

def create_dataset(df, label_name = 'ArgumentLevel', label_number = 5, batch_size = 16, max_length = 256):

    # Tokenize text
    X_input_ids = np.zeros((len(df), max_length))
    X_attn_masks = np.zeros((len(df), max_length))

    for i, text in tqdm(enumerate(df['Content'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length = max_length,
            truncation = True,
            padding = 'max_length',
            add_special_tokens = True, #add [CLS] [PAD] [SEP] tokens
            return_tensors = 'tf'
        )
        
        X_input_ids[i,:] = tokenized_text.input_ids
        X_attn_masks[i,:] = tokenized_text.attention_mask
    

    # Generate labels - One-hot encoding
    labels = np.zeros((len(df), label_number)) 
    labels[np.arange(len(df)), df[label_name].values] = 1


    # Create dataset object
    dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

    # Map dataset
    dataset = dataset.map(DatasetMapFunction)

    # shuffle data
    dataset = dataset.shuffle(400,reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True) # drop remainder 179/16 = 11...3, drop 3 data

    return dataset

Split training and validation data 

In [ ]:
def split_dataset(dataset, p = 0.5, batch_size = 16):
    # p - training data size %
    train_size = int((len(df)//batch_size)*p)
    
    train_dataset = dataset.take(train_size) # take the first 8 batches
    val_dataset = dataset.skip(train_size) # take the last 3 batches

    return train_dataset, val_dataset

Create model

In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
def create_model(label_number = 5, max_length = 256):

    # input layer
    input_ids = tf.keras.layers.Input(shape = (max_length), name = 'input_ids', dtype = 'int32')
    # attention layer
    attention_masks = tf.keras.layers.Input(shape = (max_length), name = 'attention_mask', dtype = 'int32')

    # bert layer
    bert_layer = bert_model.bert(input_ids, attention_mask=attention_masks)[1] 

    # intermediate layer
    intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name = 'intermediate_layer')(bert_layer)

    # output layer - 5 layers because 5 label classes
    output_layer = tf.keras.layers.Dense(label_number, activation='softmax', name = 'output_layer')(intermediate_layer)

    # Create model object
    model = tf.keras.Model(inputs=[input_ids, attention_masks], outputs=output_layer)



    # Add optimizer & Loss function & Accuracy metrics
    optim = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss_func = tf.keras.losses.CategoricalCrossentropy()
    acc = tf.keras.metrics.CategoricalAccuracy('accuracry')

    model.compile(optimizer = optim, loss = loss_func, metrics = [acc])

    return model


========= 分界线 ==========
==========================

In [ ]:
year = 'Y1Y2'
label_name = 'ReasoningLevel'
label_number = 4

run_num = 1

In [ ]:
folderpath = '/content/drive/MyDrive/Imperial/nlp-physicseducation/outputs'
outputpath = '/content/drive/MyDrive/Imperial/BERT-results'
filepath = '/sections/labels_cleaned_{year}.csv'.format(year = year)

Load data

In [ ]:
df = pd.read_csv(folderpath+filepath)
# df.head()
df['ArgumentLevel'] = df['ArgumentLevel'].replace({'superficial': 0, 'extended': 1, 'deep': 2, 'expert': 3, 'prediction': 4})
df['ReasoningLevel'] = df['ReasoningLevel'].replace({'bal': 0, 'the': 1, 'exp': 2, 'none': 3})
# df['ReasoningLevel'].value_counts()

/Users/jiayangzhang/Library/CloudStorage/GoogleDrive-jiayang.zhang@icloud.com/My Drive/Imperial/nlp-physicseducation/outputs/sections

In [ ]:
s=8 # train ratio

for i in range(run_num):
    # about dataset
    dataset = create_dataset(df, label_name = label_name, label_number = label_number, batch_size = 16, max_length=512) # create dataset
    train_dataset, val_dataset = split_dataset(dataset, p = (0.1*s), batch_size = 16) # split training and validation data 


    # about model
    model = create_model(label_number = label_number, max_length = 512)   # Create model


    # about train and validate
    history = model.fit(     # Train & Validate model (Fine-tuning)
        train_dataset,
        validation_data = val_dataset,
        epochs = 4
    )


    history_saveto = outputpath + '/Gradient-Descent/history/{time}_{year}_{label_name}_trainsize0{size}_4epochs_16batchsize_1e-5lr.npy'.format(
        time = datetime.now().strftime("%H%M%S"), 
        year = year, 
        label_name = label_name, 
        size = s
        )
    np.save(history_saveto, history.history) # save history


0it [00:00, ?it/s]

Epoch 1/4
8/8 [==============================] - 46s 646ms/step - loss: 1.5200 - accuracry: 0.3984 - val_loss: 1.3407 - val_accuracry: 0.5208
Epoch 2/4
8/8 [==============================] - 3s 323ms/step - loss: 1.3444 - accuracry: 0.4688 - val_loss: 1.2836 - val_accuracry: 0.4167
Epoch 3/4
8/8 [==============================] - 3s 331ms/step - loss: 1.2000 - accuracry: 0.4688 - val_loss: 1.1008 - val_accuracry: 0.6250
Epoch 4/4
8/8 [==============================] - 3s 326ms/step - loss: 1.0879 - accuracry: 0.5938 - val_loss: 1.0208 - val_accuracry: 0.7083


Save model

In [ ]:
model_saveto = outputpath + '/Gradient-Descent/model/{time}_{year}_{label_name}_trainsize0{size}_4epochs_16batchsize_1e-5lr'.format(
        time = datetime.now().strftime("%H%M%S"), 
        year = year, 
        label_name = label_name, 
        size = s
        )
model.save(model_saveto)